In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:



!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.1 MB/s eta 0:00:00


In [ ]:

from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import tensorflow as tf
import string
import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import ast

import tensorflow_hub as hub
from transformers import ViltProcessor, ViltForImagesAndTextClassification
import numpy as np
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from pathlib import Path

from tqdm.notebook import tqdm
import albumentations as albu
from albumentations import HorizontalFlip, VerticalFlip, Rotate
import imageio
from glob import glob
from PIL import Image

In [ ]:

train_df = pd.read_csv("/content/drive/MyDrive/PFA/DATASET/trainset.csv",encoding='ISO-8859-1')
valid_df = pd.read_csv("/content/drive/MyDrive/PFA/DATASET/validationset.csv",encoding='ISO-8859-1')
test_df =pd.read_csv("/content/drive/MyDrive/PFA/DATASET/testset.csv",encoding='ISO-8859-1')

In [ ]:
train_df["Medias"]= ast.literal_eval((strtrain_df["Medias"]))
train_df.dropna(subset=["Medias"], inplace=True)
valid_df["Medias"]= ast.literal_eval(str(valid_df["Medias"]))
valid_df.dropna(subset=["Medias"], inplace=True)
test_df["Medias"]= ast.literal_eval(str(test_df["Medias"]))
test_df.dropna(subset=["Medias"], inplace=True)


ValueError: ignored

In [ ]:
train_df['Medias_len'] = train_df['Medias'].apply(len)
train_df = train_df.drop(train_df[train_df['Medias_len'] < 2].index)
train_df = train_df.drop('Medias_len', axis=1)

In [ ]:

valid_df['Medias_len'] = valid_df['Medias'].apply(len)
valid_df = valid_df.drop(valid_df[valid_df['Medias_len'] < 2].index)
valid_df = valid_df.drop('Medias_len', axis=1)

In [ ]:
test_df['Medias_len'] = test_df['Medias'].apply(len)
test_df = test_df.drop(test_df[test_df['Medias_len'] < 2].index)
test_df = test_df.drop('Medias_len', axis=1)

In [ ]:
transform = transforms.ToTensor()

In [ ]:
class ImageFolderCustom(torch.utils.data.Dataset):
    transform = transform

    def __init__(self,df ):
        self.images = df['Medias']
        self.captions = df['text_clean']
        self.label = df['I-E']
    
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, index):
        paths = ast.literal_eval(self.images[index])[0:2]
        img1 = Image.open(paths[0])
        img2 = Image.open(paths[1])
        captions = ast.literal_eval(self.captions[index])[0:2]
        text = ''.join(captions)
        
        img1 = self.transform(img1)
        img2 = self.transform(img2)
        if self.label[index] == 0:
          label =True
        else:
          label = False
        label = torch.tensor(label)
       
        return img1,img2,text,label

In [ ]:
train_dataset = ImageFolderCustom(train_df)

train_dataloader = DataLoader(dataset=train_dataset, 
                              batch_size=1, 
                              num_workers=0, 
                              shuffle=True)

In [ ]:
test_dataset = ImageFolderCustom(test_df)


test_dataloader = DataLoader(dataset=test_dataset, 
                              batch_size=1, 
                              num_workers=0, 
                              shuffle=True)

In [ ]:
img1,img2,text,label = next(iter(train_dataloader))

In [ ]:
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-nlvr2")
model = ViltForImagesAndTextClassification.from_pretrained("dandelin/vilt-b32-finetuned-nlvr2")

In [ ]:
encoding = processor([img1[0],img2[0]],text, return_tensors="pt")


In [ ]:
outputs = model(input_ids=encoding.input_ids, pixel_values=encoding.pixel_values.unsqueeze(0))
logits = outputs.logits
idx = logits.argmax(-1).item()
print("Predicted answer:", model.config.id2label[idx])

Predicted answer: True


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

cuda:0


In [ ]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device):
    model.train()
  
    train_loss= 0
  
    for batch,(img1,img2,text,label) in enumerate(dataloader):
        im1 = img1.to(device)
        im2 = img2.to(device)
        float_label = float(label)
        encoding = processor([img1[0],img2[0]],text, return_tensors="pt")
        outputs = model(input_ids=encoding.input_ids, pixel_values=encoding.pixel_values.unsqueeze(0))
        

        logits = outputs.logits
        pred_traget = torch.tensor(logits.max().item(), device=label.device,  dtype=torch.float64)
        idx = logits.argmax(-1).item()
        label_idx =  torch.tensor(label.argmax(-1).item(), device=label.device,  dtype=torch.float64)
        pred = model.config.id2label[idx]
        if pred =='True':
            pred = True
        else:

            pred = False
        print('pred',pred)
        float_pred = float(pred)

        loss = loss_fn(torch.tensor([float_pred]), torch.tensor([float_label]))
       
        print("loss : ",loss)
        loss.backward()

        optimizer.step()
        train_loss += loss.item() 
        ########
        break


    train_loss = train_loss / len(dataloader)
    return train_loss

In [ ]:
def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device):
    model.eval() 
  
    test_loss = 0
    with torch.no_grad():
          for batch,(img1,img2,text,label) in enumerate(dataloader):
              img1 = img1.to(device)
              img2 = img2.to(device)
              text = text
          
              print('label',label)
              float_label = float(label)
              encoding = processor([img1[0],img2[0]],text, return_tensors="pt")
              outputs = model(input_ids=encoding.input_ids, pixel_values=encoding.pixel_values.unsqueeze(0))
              logits = outputs.logits
              idx = logits.argmax(-1).item()
              pred = model.config.id2label[idx]
              if pred =='True':
                pred = True
              else:

                pred = False
              float_pred = float(pred)
            

              loss = loss_fn(torch.tensor([float_pred]), torch.tensor([float_label]))
              print(loss)
              test_loss += loss.item() 


    test_loss = test_loss / len(dataloader)
    return test_loss

 

In [ ]:
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs,
          device):
    
    # Create empty results dictionary
    results = {"train_loss": [],
      "test_loss": []
    }
  
    for epoch in tqdm(range(epochs)):
        train_loss = train_step(model=model,
                                dataloader=train_dataloader,
                                loss_fn=loss_fn,
                                optimizer=optimizer,
                                device=device)
        
        test_loss = test_step(model=model,
                              dataloader=test_dataloader,
                              loss_fn=loss_fn,
                              device=device)
      
        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"test_loss: {test_loss:.4f} | "
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)

    # Return the filled results at the end of the epochs
    return results

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
loss = nn.BCEWithLogitsLoss()

In [ ]:
model_results = train(model=model, 
      train_dataloader=train_dataloader,
      test_dataloader=train_dataloader,
     optimizer=optimizer,
     loss_fn=loss,
     epochs=5,
     device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

pred True
loss :  tensor(-0.)


RuntimeError: ignored

In [ ]:
float_label = 1
pred = False
float_pred = float(pred)
p_tens = torch.tensor([float_pred])
l_tens = torch.tensor([float_label])

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss.backward()

optimizer.step()
train_loss += loss.item() 